In [0]:
from pyspark.sql.functions import lit
from functools import reduce

In [0]:
# Lista de arquivos por mês
months = ["01", "02", "03", "04", "05"]
paths = [f"s3://case-ifood-matsuura/newyork/landing/yellow_tripdata_2023-{m}.parquet" for m in months]

# Capturar o schema base
base_df = spark.read.parquet(paths[0])
base_columns = base_df.columns

# Lista final de dataframes ajustados
bronze_df_list = []

for path in paths:
    df = spark.read.parquet(path)
    
    # Adiciona colunas ausentes com null
    for col_name in base_columns:
        if col_name not in df.columns:
            df = df.withColumn(col_name, lit(None))
    
    # Garante que a ordem das colunas fique igual
    df = df.select(base_columns)
    
    bronze_df_list.append(df)

# Juntando tudo
bronze_df = reduce(lambda df1, df2: df1.unionByName(df2, allowMissingColumns=True), bronze_df_list)

# Grava como tabela Delta
bronze_df.write.format("delta").mode("overwrite").saveAsTable("newyork.bronze.yellow_taxi")


In [0]:
df = spark.read.parquet("s3a://case-ifood-matsuura/newyork/landing/yellow_tripdata_2023-01.parquet")
df.printSchema()